In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
# Load the data
df_salary = pd.read_csv("./datasets/salary_dataset.csv")

In [4]:
# Inspect the structure of the data
df_salary.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   tempo_na_empresa  1000 non-null   int64  
 1   nivel_na_empresa  1000 non-null   int64  
 2   salario_em_reais  1000 non-null   float64
dtypes: float64(1), int64(2)
memory usage: 23.6 KB


In [5]:
# Copy DF for DF EDA
df_salary_eda = df_salary.copy()

In [ ]:
# Copy DF for DF Bucketing
df_salary_bucketing = df_salary.copy()